通信確認

In [1]:
import subprocess

commands = ["ping", "-n", "2", "-w", "7777", "150.12.70.113"]
print(" ".join(commands))
proc = subprocess.run(
    commands,
    stdout=subprocess.PIPE,     # 標準出力は保存
    stderr=subprocess.DEVNULL   # 標準エラーは捨てる
)
print(f"return code : {proc.returncode}")
result = proc.stdout.decode("cp932")
print(result)
print()

ping -n 2 -w 7777 150.12.70.113



return code : 0







150.12.70.113 に ping を送信しています 32 バイトのデータ:



150.12.70.113 からの応答: バイト数 =32 時間 =32ms TTL=64



150.12.70.113 からの応答: バイト数 =32 時間 =3ms TTL=64







150.12.70.113 の ping 統計:



    パケット数: 送信 = 2、受信 = 2、損失 = 0 (0% の損失)、



ラウンド トリップの概算時間 (ミリ秒):



    最小 = 3ms、最大 = 32ms、平均 = 17ms










テスラメーター読み取り

In [1]:
import room138.module138.tesla
room138.module138.tesla.tesla("C:/Users/wadat/Data/2021_10_27@tesla_curr_calibration/a.tesla.txt")

SerialException: could not open port 'COM3': FileNotFoundError(2, '指定されたファイルが見つかりません。', None, 2)

電源に電圧の入力・読み取り

In [1]:
import room138.module138.curr

room138.module138.curr.curr0("C:/Users/wadat/Data/2021_10_27@tesla_curr_calibration/a.curr.txt")
for i in range(1, 4):
    j=format(i*0.1, '.1f')
    room138.module138.curr.curr("C:/Users/wadat/Data/2021_10_27@tesla_curr_calibration/a.curr.txt",j)

+0.00000E+00\



+1.00000E-01\n



+2.00000E-01\n



+3.00100E-01\n


◎電圧・磁場キャリブレーション

In [ ]:
import room138.module138.tesla
import room138.module138.curr
import os

date = "2021_11_16"
num  = "1"

#######################################################################

curr   = 'C:/Users/wadat/Data/' + date + '/curr_log/'    #電流値
tesla  = 'C:/Users/wadat/Data/' + date + '/tesla_log/'   #磁場

curr_path  =  curr + num + '.txt' 
tesla_path = tesla + num + '.txt' 

os.makedirs(curr,  exist_ok=True)
os.makedirs(tesla, exist_ok=True)

room138.module138.KIKUSUI.currOFF()
room138.module138.KIKUSUI.curr0(curr_path)
room138.module138.tesla.tesla(tesla_file)

room138.module138.KIKUSUI.currON()

for i in range(1, 31):
    A = format(i*0.1, '.1f')
    room138.module138.KIKUSUI.curr(curr_file,A)
    room138.module138.tesla.tesla(tesla_file)
    
room138.module138.KIKUSUI.teslaON()

for i in reversed(range(1, 31)):
    A = format(i*0.1, '.1f')
    room138.module138.KIKUSUI.curr(curr_file,A)
    room138.module138.tesla.tesla(tesla_file)

room138.module138.KIKUSUI.currOFF()
room138.module138.KIKUSUI.teslaOFF()

ツジコンモーター移動

In [7]:
import room138.module138.λ
room138.module138.λ.move(14521)

OK!


ツジコンモーター位置読み出し

In [3]:
import room138.module138.λ
room138.module138.λ.status("C:/Users/wadat/Data/2021_10_27@tesla_volt_calibration/a.status.txt")

+0000200


ツジコンモーター移動

In [5]:
import module138.λ
module138.λ.move(78801)

OK!


◎ゼロ点探し

In [ ]:
import room138.module138.λ
import room138.module138.pico
import os

date = "2021_11_16"
num  = "1"

range_start = 65168   #モーター移動開始
range_stop  = 165169  #モーター移動開始
range_step  = 2000    #刻み
detect_time = 10      #積算回数

################################################################################

status = 'C:/Users/wadat/Data/' + date + '/status_log/'  #モーター位置
pico1  = 'C:/Users/wadat/Data/' + date + '/pico1_log/'   #ピコアン１
pico2  = 'C:/Users/wadat/Data/' + date + '/pico2_log/'   #ピコアン２
pico3  = 'C:/Users/wadat/Data/' + date + '/pico3_log/'   #差分電圧（ピコアン１−２）

status_path  = status + num + '.txt' 
pico_path1   =  pico1 + num + '.txt' 
pico_path2   =  pico2 + num + '.txt' 
pico_path3   =  pico3 + num + '.txt' 

os.makedirs(status, exist_ok=True)
os.makedirs(pico1,  exist_ok=True)
os.makedirs(pico2,  exist_ok=True)
os.makedirs(pico3,  exist_ok=True)

for i in range(range_start, range_stop, range_step):
    module138.λ.move(i)
    module138.λ.status(status_path)
    i = 0
    while i < detect_time:
        module138.pico.pico1(pico_path1)
        module138.pico.pico2(pico_path2)
        i += 1
        
room138.module138.pico.diffpico(pico_path1,pico_path2,pico_path3)

◎M-H曲線

In [5]:
import room138.module138.KIKUSUI
import room138.module138.pico
import os

def add(y):
    x = y / 42.717　#磁場・電流の係数
    return x

date = "2021_11_16"
num  = "1"

range_start = 10   #測定開始磁場
range_stop  = 120  #測定終了磁場
range_step  = 10   #刻み
detect_time = 10   #積算回数

##################################################################################

curr  = 'C:/Users/wadat/Data/' + date + '/curr_log/'    #電流値
pico1 = 'C:/Users/wadat/Data/' + date + '/pico1_log/'   #ピコアン１
pico2 = 'C:/Users/wadat/Data/' + date + '/pico2_log/'   #ピコアン２
pico3 = 'C:/Users/wadat/Data/' + date + '/pico3_log/'   #差分電圧（ピコアン１−２）

curr_path  =  curr + num + '.txt' 
pico_path1 = pico1 + num + '.txt' 
pico_path2 = pico2 + num + '.txt' 
pico_path3 = pico3 + num + '.txt' 

os.makedirs(curr,  exist_ok=True)
os.makedirs(pico1, exist_ok=True)
os.makedirs(pico2, exist_ok=True)
os.makedirs(pico3, exist_ok=True)

################### 上り #######################

room138.module138.KIKUSUI.currOFF()
room138.module138.KIKUSUI.curr0(curr_path)

i = 0
while i < detect_time:
    room138.module138.pico.pico1(pico_path1)
    room138.module138.pico.pico2(pico_path2)
    i += 1

room138.module138.KIKUSUI.currON()

for i in range(range_start,range_stop,range_step):
    x = add(i)
    room138.module138.KIKUSUI.curr(curr_path,x)
    i = 0
    while i < detect_time:
        room138.module138.pico.pico1(pico_path1)
        room138.module138.pico.pico2(pico_path2)
        i += 1
        
for i in reversed(range(range_start,range_stop,range_step)):
    x = add(i)
    room138.module138.KIKUSUI.curr(curr_path,x)
    i = 0
    while i < detect_time:
        room138.module138.pico.pico1(pico_path1)
        room138.module138.pico.pico2(pico_path2)
        i += 1

room138.module138.KIKUSUI.currOFF()
room138.module138.KIKUSUI.curr0(curr_path)
i = 0
while i < detect_time:
    room138.module138.pico.pico1(pico_path1)
    room138.module138.pico.pico2(pico_path2)
    i += 1
    
################### 下り #######################
    
room138.module138.KIKUSUI.teslaON()
room138.module138.KIKUSUI.currON()

for i in range(range_start,range_stop,range_step):
    x = add(i)
    room138.module138.KIKUSUI.curr(curr_path,x)
    i = 0
    while i < detect_time:
        room138.module138.pico.pico1(pico_path1)
        room138.module138.pico.pico2(pico_path2)
        i += 1
        
for i in reversed(range(range_start,range_stop,range_step)):
    x = add(i)
    room138.module138.KIKUSUI.curr(curr_path,x)
    i = 0
    while i < detect_time:
        room138.module138.pico.pico1(pico_path1)
        room138.module138.pico.pico2(pico_path2)
        i += 1

room138.module138.KIKUSUI.currOFF()
room138.module138.KIKUSUI.curr0(curr_path)
i = 0
while i < detect_time:
    room138.module138.pico.pico1(pico_path1)
    room138.module138.pico.pico2(pico_path2)
    i += 1
    
room138.module138.KIKUSUI.teslaOFF()
room138.module138.pico.diffpico(pico_path1,pico_path2,pico_path3)

+0.00000E+00\n

7.471539E-01\n

8.232352E-01\n

7.471904E-01\n

8.232800E-01\n

7.471492E-01\n

8.233244E-01\n

7.471881E-01\n

8.232784E-01\n

7.471197E-01\n

8.231693E-01\n

7.471411E-01\n

8.232722E-01\n

7.472070E-01\n

8.233876E-01\n

7.473038E-01\n

8.232560E-01\n

7.472992E-01\n

8.232762E-01\n

7.471873E-01\n

8.232957E-01\n

+2.34300E-01\n

7.476049E-01\n

8.227197E-01\n

7.476639E-01\n

8.227846E-01\n

7.475373E-01\n

8.226675E-01\n

7.476433E-01\n

8.227822E-01\n

7.475071E-01\n

8.227136E-01\n

7.473916E-01\n

8.228380E-01\n

7.474915E-01\n

8.227559E-01\n

7.474597E-01\n

8.227287E-01\n

7.475319E-01\n

8.227486E-01\n

7.474985E-01\n

8.227324E-01\n

+4.68400E-01\n

7.478699E-01\n

8.223294E-01\n

7.479375E-01\n

8.222367E-01\n

7.478179E-01\n

8.222585E-01\n

7.478346E-01\n

8.221957E-01\n

7.479023E-01\n

8.222117E-01\n

7.478416E-01\n

8.222141E-01\n

7.479071E-01\n

8.221736E-01\n

7.479341E-01\n

8.222817E-01\n

7.479779E-01\n

8.222608E-01\n

7.478684E-01\n

8.223282

NameError: name 'linecache' is not defined